# 誤差逆伝播法

「誤差伝播法ってなぁに？」  
「ニューラルネットワークの学習時に、パラメータの勾配を求める為の手法だよ」

## 合成関数の微分

高校でやるやつ。これが分かれば誤差逆伝播法なんてほとんど理解したようなもん

<br>

試しに、以下の関数を微分してみよう。

$$
y = (x + 1)^2
$$

これは普通に展開しても解けるけど、合成関数の微分を使っても解けるね

$$
u = x + 1 \\
y = u^2 \\
\frac{dy}{dx} = \frac{dy}{du} \frac{du}{dx} = 2u \cdot 1 = 2(x + 1)
$$

こんな感じで、**関数の関数**(合成関数)を微分するとき、関数ごとに微分をしたものをかけ合わせればよかった

<br>

じゃあここまでの流れをPythonで実装してみよう

以下の二つの関数を`class`として実装する。
- $f(x) = x + 1$
- $g(x) = x^2$

`class`にする必要ある？関数でよくね？と思うかもしれんが、まあ読んでみてよ。  
まずは$f(x) = x + 1$から

In [2]:
class Plus1:
    def __call__(self, x):
        return x + 1

出来た。入力した値に1を足して出力するだけの関数。`__call__()`というのは特殊メソッドで、関数のように`()`をつけて呼び出したときに実行されるヤツ。  
こんな感じ

In [15]:
plus1 = Plus1() # インスタンス生成
y = plus1(3) # 関数の呼び出し
print(y)

4


入力した3に1を足した4が出力された。

では、今度は微分を行うメソッドを書いてみよう。「微分を行う」というのを、「入力された値に微分した値をかけて出力する」と捉えるとこうなる

In [20]:
class Plus1:
    def __call__(self, x):
        return x + 1

    def backward(self, d):
        return d * 1

`backward()`というメソッドを追加した。これは逆伝播という意味。  
$x + 1$を$x$で微分すると1になるので、入力値(d)に1をかけて出力させる。

こんなノリで、$g(x)$の方も書いちゃおう

In [21]:
class Square:
    def __call__(self, x):
        self.x = x
        return x ** 2

    def backward(self, d):
        return d * self.x*2

できた。$g(x) = x^2$を微分すると$2x$になるので、それを入力値(d)にかけて出力する。微分するときに使うので、初めに入力された値(x)は変数に保存しておく。

ではこれらを使って、実際に計算してみよう。  
ここから、

$$
h(x) = (x + 1)^2
$$

とおく

In [18]:
# インスタンス生成
plus1 = Plus1()
square = Square()

# 計算
x = 3
u = plus1(x)
y = square(u)
print(y)

16


でた。$h(3) = 16$ということで、正解！

じゃあ今度は$h$の$x=3$での傾きを求めてみよう。$h'(3)$のことだね。  
そしてこれはこんな感じで求められる

In [19]:
d = square.backward(1)
d = plus1.backward(d)
print(d)

8


$$
h'(x) = 2(x + 1) \\
h'(3) = 8
$$

ということで正解！  
これは合成関数の微分に基づいていて、正に「関数ごとの微分を掛け合わせる」という部分に当たる。

> 入力された値に微分した値をかけて出力する

さっきこう捉えた意味が分かったかな...?  
微分した結果を後ろの方に伝えていく感じだね。一番初めは1を入力しておく。

<br>

こんな感じで、複数の関数を経て出力された値を何らかの変数で微分した値は、関数ごとに微分をすれば簡単に求まる。
んで、これはニューラルネットワークが持つパラメータの勾配を求めるときにも使える。

損失をパラメータで微分した値(勾配)を求めるとき、損失を出す際に通った**層**や**損失関数**を一つ一つ微分すればいいよねという話。

## ニューラルネットワークの構築

ということで、誤差逆伝播法で学習を行うニューラルネットワークを作ってみようじゃないか

In [22]:
import numpy as np

## 層の定義

NNは複数の層から構成されるので、まずは層を作る。

#### ReLU

パラメータを持つ層は工夫が必要なので、一旦パラメータを持たない層を作ってみよう。  
さっきの関数と同じように作ればOK

ちなみにReLUはこういう関数

$$
y = \begin{cases}
x & (x > 0) \\
0 & (x \leq 0) \\
\end{cases}
$$

In [23]:
class ReLU:
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, d):
        return d * (self.x > 0)

    def update(self, lr):
        pass

できた。先程実装したような一般的な関数の入出力は「数値」だが、NNの層の入出力は「ベクトル（というかテンソル）」で行うのでnumpyを使う。  
あと、後々のことを考えて、パラメータを更新するメソッド`update()`を書いている。ReLUはパラメータを持たないので何もしないけど

#### 全結合層

全結合層。kerasでいうDense。PyTorchでいうLinear。ここではPyTorchに倣ってLinearにしよー

In [ ]:
class Linear:
    def __init__(self, n_input, n_output):
        self.w = np.random.randn(n_input, n_output)
        self.b = np.random.randn(n_output)

    def __call__(self, x):
        self.x = x
        return np.dot(x, self.w) + self.b

    def backward(self, d):
        self.grad_w = np.dot(self.x.T, d)
        self.grad_b = np.sum(d, axis=0)
        return np.dot(d, self.w.T)

    def update(self, lr):
        self.w -= lr * self.grad_W
        self.b -= lr * self.grad_b

### 損失関数